In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('data_decomposed.csv')

In [2]:
from sklearn import metrics
def print_metrics(pred, y_vals):
    print('mape: ', metrics.mean_absolute_percentage_error(y_vals, pred))
    print('mae: ', metrics.mean_absolute_error(y_vals, pred))
    print('mse: ', metrics.mean_squared_error(y_vals, pred))
    print('rmse: ', np.sqrt(metrics.mean_squared_error(y_vals, pred)))
    print('r2: ', metrics.r2_score(y_vals, pred))
    count = 0
    y_error = pred.flatten() - y_vals.flatten()
    y_error = np.array([abs(e) for e in y_error]).flatten()
    for i in range(len(y_error)):
        if(y_error[i] < 0.15 * y_vals[i]):
            count += 1
    print('15% 准确度: ', count / len(pred))

In [3]:
import os
split_line = 5828
window = 8
y_row_list = []
weather_features = ["lrad", "pres", "shum", "srad", "temp", "wind"]
for row in range(split_line-window):
    y_row_list.append(dict(("y"+weather_feature,data[weather_feature][row+window]) for weather_feature in weather_features))
y=pd.DataFrame(y_row_list)

In [4]:
from sklearn.preprocessing import StandardScaler
yscaler = StandardScaler()
y_train = yscaler.fit_transform(y.to_numpy())
pd.DataFrame(y_train)

,0,1,2,3,4,5,6
0,-1.346956,-0.104181,0.952739,-0.559640,-0.720365,-1.414993,-0.458198
1,-1.211764,-0.104181,1.060847,-0.466545,0.176453,-1.231573,-0.839920
2,-0.775288,-0.104181,0.788939,-0.419997,0.738248,-0.542844,-0.168938
3,-0.713486,-0.104181,0.684107,-0.605436,-0.251892,-0.401473,0.449451
4,-1.111336,-0.104181,0.720143,-0.596802,-0.721299,-1.011905,-0.246130
...,...,...,...,...,...,...,...
5815,-1.346956,-0.104181,1.093607,-1.059650,-0.721299,-1.955101,-1.362453
5816,-1.725492,-0.104181,1.136195,-0.920007,-0.720365,-2.062398,-1.693278
5817,-1.570988,-0.104181,1.273787,-0.999213,0.139124,-1.720933,-1.415894
5818,-1.354681,-0.104181,1.024811,-0.868205,0.704652,-1.346845,-1.435405


In [5]:
from prophet import Prophet
filename='all_feature_prophet_gru.npy'
features = ['trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper', 'additive_terms',
            'additive_terms_lower', 'additive_terms_upper', 'yhat']
if os.path.isfile(filename):
    X=np.load(filename)
else:
    X=np.empty((0,(len(features)+1)*window*len(weather_features)))
    for row in range(split_line-window):
        temp_array=np.empty((1,0))
        for weather_feature in weather_features:
            temp_df = pd.DataFrame(data['ds'][row:row+window])
            temp_df['y'] = data[weather_feature][row:row+window]
            m = Prophet(changepoint_prior_scale=1.0, seasonality_prior_scale=0.1, seasonality_mode='additive', changepoint_range=1, n_changepoints=window-1)
            m.fit(temp_df)
            future = m.make_future_dataframe(periods=1, freq='3H')
            forecast = m.predict(future)
            forecast = forecast[features].to_numpy()[:-1]
            forecast = np.append(forecast, np.array([data[weather_feature][row+i] for i in range(window)]).reshape(-1,1), axis=1)
            forecast = forecast.flatten()
            forecast = forecast.reshape(1,-1)
            temp_array=np.append(temp_array, forecast, axis=1)
        X=np.append(X, temp_array, axis=0)
    np.save(filename, X)
X

array([[1.90750000e+02, 1.90750000e+02, 1.90750000e+02, ...,
        0.00000000e+00, 2.95564318e+00, 5.15199660e+00],
       [2.02000000e+02, 2.02000000e+02, 2.02000000e+02, ...,
        0.00000000e+00, 2.94914557e+00, 2.96399700e+00],
       [2.12750000e+02, 2.12750000e+02, 2.12750000e+02, ...,
        0.00000000e+00, 2.98501904e+00, 2.06399540e+00],
       ...,
       [1.72500000e+02, 1.72500000e+02, 1.72500000e+02, ...,
        0.00000000e+00, 5.19981400e-02, 5.19981400e-02],
       [1.95750000e+02, 1.95750000e+02, 1.95750000e+02, ...,
        0.00000000e+00, 7.05997470e-01, 7.05997470e-01],
       [1.92250000e+02, 1.92250000e+02, 1.92250000e+02, ...,
        0.00000000e+00, 6.59996030e-01, 6.59996030e-01]])

In [6]:
X.shape

(5820, 560)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA

In [8]:
import tensorflow as tf
from tensorflow import keras
tf.config.set_visible_devices([], 'GPU')
modelname='all_feature_prophet_gru'
scaler_list=[]
pca_list=[]
data_list=[]
model_list=[]
for weather_feature_index in range(len(weather_features)):
    modelfname=modelname+ '_' + weather_features[weather_feature_index]
    scaler = StandardScaler()
    scaler_list.append(scaler)
    X_train = scaler.fit_transform(X.reshape(split_line-window,len(weather_features),window,-1)[:,weather_feature_index].reshape(split_line-window,-1))
    pca = KernelPCA(kernel='rbf')
    pca_list.append(pca)
    X_train = pca.fit_transform(X_train)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    data_list.append(X_train)
    if os.path.isdir(modelfname):
        model = keras.models.load_model(modelfname)
    else:
        model = keras.models.Sequential()
        model.add(keras.layers.GRU(1024,input_shape = (1, X_train.shape[2]),return_sequences = True))
        model.add(keras.layers.GRU(512,activation = 'gelu', recurrent_activation = 'gelu',return_sequences = True))
        model.add(keras.layers.Dense(1))
        es_callback = keras.callbacks.EarlyStopping(patience = 5, restore_best_weights = True, monitor="loss")
        model.compile(loss = keras.losses.Huber(), optimizer = keras.optimizers.Nadam(0.001))
        model.summary()
        keras.utils.plot_model(model, to_file=modelfname+'model_plot.pdf', show_shapes=True, show_layer_names=True)
        print(weather_features[weather_feature_index])
        history=model.fit(X_train, y_train[:,weather_feature_index].reshape(-1,1), epochs = 10, verbose = 1, shuffle = True, callbacks = [es_callback])
        import matplotlib.pyplot as plt
        %matplotlib notebook
        plt.rcParams['font.size']=64
        plt.rcParams['font.sans-serif']=['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
        pic = plt.figure(figsize=(32,32))
        plt.plot(history.history['loss'])
        plt.title(u'使用多特征的Prophet-GRU-NN模型训练损失变化-GRU-'+weather_features[weather_feature_index])
        plt.xlabel(u'Epoch')
        plt.ylabel(u'损失')
        pic.savefig(modelfname + "_training_loss.pdf")
        model.save(modelfname)
    model_list.append(model)

(5820, 80)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 1024)           21027840  
                                                                 
 gru_1 (GRU)                 (None, 1, 512)            2362368   
                                                                 
 dense (Dense)               (None, 1, 1)              513       
                                                                 
Total params: 23,390,721
Trainable params: 23,390,721
Non-trainable params: 0
_________________________________________________________________
wind
Epoch 1/10
182/182 [==============================] - 100s 526ms/step - loss: 0.4006
Epoch 2/10
 44/182 [======>.......................] - ETA: 1:10 - loss: 0.3876

KeyboardInterrupt: 

In [ ]:
row_list = y_row_list.copy()
for pointer in range(X.shape[0],len(data)-split_line+X.shape[0]):
    gru_output=np.empty((1,1,0))
    for weather_feature_index in range(len(weather_features)):
        input_np = np.array([row_list[row]['y'+weather_features[weather_feature_index]] for row in range(pointer-window, pointer)])
        temp_df = pd.DataFrame(data['ds'][pointer:pointer+window])
        temp_df['y'] = input_np
        m = Prophet(changepoint_prior_scale=1.0, seasonality_prior_scale=0.1, seasonality_mode='additive', changepoint_range=1, n_changepoints=window-1)
        m.fit(temp_df)
        future = m.make_future_dataframe(periods=1, freq='3H')
        forecast = m.predict(future)
        forecast = forecast[features].to_numpy()[:-1]
        forecast = np.append(forecast, np.array([row_list[row]['y'+weather_features[weather_feature_index]] for row in range(pointer-window, pointer)]).reshape(-1,1), axis=1)
        forecast = forecast.flatten().reshape(1,-1)
        input_np = scaler_list[weather_feature_index].transform(forecast)
        input_np = pca_list[weather_feature_index].transform(input_np)
        input_np = np.reshape(input_np, (input_np.shape[0], 1, input_np.shape[1]))
        output_np = model_list[weather_feature_index].predict(input_np)
        gru_output=np.append(gru_output, output_np, axis=2)
    row_list.append(dict(("y"+weather_features[index],yscaler.inverse_transform(gru_output)[:,:,index][0][0]) for index in range(len(weather_features))))

In [ ]:
predictions = yscaler.transform(pd.DataFrame(row_list)[X.shape[0]:].to_numpy()).reshape(-1,1,len(weather_features))
predictions.shape

In [ ]:
y_train = y.to_numpy()
y_train.shape

In [ ]:
gru_output=np.empty((X.shape[0],1,0))
for index in range(len(data_list)):
    predicted = model_list[index].predict(data_list[index])
    predicted = predicted[:,:,-1].reshape(X.shape[0],1,-1)
    gru_output=np.append(gru_output, predicted, axis=2)
gru_output.shape

In [ ]:
if os.path.isdir(modelname):
    model = keras.models.load_model(modelname)
else:
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=(1, len(weather_features))))
    model.add(keras.layers.Dense(128, activation='gelu'))
    model.add(keras.layers.Dense(64, activation='gelu'))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(1))
    es_callback = keras.callbacks.EarlyStopping(patience = 50, restore_best_weights = True, monitor="loss")
    model.compile(loss = keras.losses.Huber(), optimizer = keras.optimizers.Nadam(0.001))
    model.summary()
    keras.utils.plot_model(model, to_file=modelname+'model_plot.pdf', show_shapes=True, show_layer_names=True)
    history=model.fit(gru_output, y_train.reshape(-1,1,1), epochs = 5000, verbose = 1, shuffle = True, callbacks = [es_callback])
    import matplotlib.pyplot as plt
    %matplotlib notebook
    plt.rcParams['font.size']=64
    plt.rcParams['font.sans-serif']=['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    pic = plt.figure(figsize=(32,32))
    plt.plot(history.history['loss'])
    plt.title(u'使用多特征的Prophet-GRU-NN模型训练损失变化-NN')
    plt.xlabel(u'Epoch')
    plt.ylabel(u'损失')
    pic.savefig(modelname + "_training_loss.pdf")
    model.save(modelname)

In [ ]:
predicted = model.predict(gru_output).flatten()
predicted[predicted < 0] = 0

predictions = model.predict(predictions).flatten()
predictions[predictions < 0] = 0

In [ ]:
print_metrics(predicted, y_train)

In [ ]:
print_metrics(predictions, data["wind"][split_line:].to_numpy())

In [ ]:
generated = pd.DataFrame(data['ds'])[window:]
generated['y_pred'] = np.append(predicted, predictions)
generated['y_real'] = np.array(pd.DataFrame(data['wind'])[window:])
generated.to_csv(modelname + ".csv")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
plt.rcParams['font.size']=64
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

fore_line=4100
end_line=4200
pic = plt.figure(figsize=(32,32))
plt.plot(y_train[fore_line:end_line],'g',predicted[fore_line:end_line],'r')
plt.title('多特征ICCEMDAN-Prophet-GRU-NN模型训练集结果')
plt.plot(y_train[fore_line:end_line],'g',label=u'真实值')
plt.plot(predicted[fore_line:end_line],'r',label=u'预测值')
plt.xlabel(u'时间（3小时）')
plt.ylabel(u'近地面全风速（m/s）')
plt.legend()
pic.savefig(modelname + "_predict_train.pdf")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
plt.rcParams['font.size']=64
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

pic = plt.figure(figsize=(32,32))
plt.plot(data["wind"][split_line:].to_numpy(),'g',predictions,'r')
plt.title('多特征ICCEMDAN-Prophet-GRU-NN模型测试集结果')
plt.plot(data["wind"][split_line:].to_numpy(),'g',label=u'真实值')
plt.plot(predictions,'r',label=u'预测值')
plt.xlabel(u'时间（3小时）')
plt.ylabel(u'近地面全风速（m/s）')
plt.legend()
pic.savefig(modelname + "_predict_test.pdf")